<h1 align="center">SETTINGS</h1> 

<h2 align="center">STEP1: Geographical selection<h2>

In [11]:
import ipywidgets as ipywd
import base64
import hashlib
from typing import Callable
from IPython.display import HTML, display
import ipyleaflet as ipylf
import geopandas as gpd
import pandas as pd
from pathlib import Path
from shapely.geometry import Polygon, MultiPolygon, MultiLineString, LinearRing
from shapely.ops import unary_union
from io import BytesIO
import yaml
#local imports
# from Shapefile import subcountrymap
from emission_explorer.Shapefile import subcountrymap

In [12]:
###########################################################################################################
# COMMON STYLES
###########################################################################################################
hover_style = {"fillColor": "#40E0D0", "fillOpacity": 0.5}
default_style = {
        "color": "black",
        "fillColor": "#366370",
        "opacity": 0.05,
        "weight": 1.9,
        "dashArray": "2",
        "fillOpacity": 0.6}
click_style = {"color": "#40E0D0",
            "fillColor": "red",#"#366370",
#             "opacity": 0.4,
            "weight": 3,
#             "dashArray": "3",
            "fillOpacity": 0.2}

In [13]:
global DEFAULT_CONFIG_FILE
global TOTAL_CONFIG
DEFAULT_CONFIG_FILE = Path('/home/esowc32/PROJECT/DATA/code_config.yml')
if not DEFAULT_CONFIG_FILE.exists():
    DEFAULT_CONFIG_FILE = Path.cwd() / "code_config.yml"
    
TOTAL_CONFIG = {'geometry':'',
                'aggregating_operation':'',
                'specific_start_date':'',
                'specific_end_date' :'',
                'reference_start_date':'',
                'reference_end_date'  :'',
                'resolution'          :'',
                'variable'            :'',
                'plot_type'           :''}

In [14]:
out = ipywd.Output()
display(out)

Output()

In [15]:
class DownloadButton(ipywd.Button):
    """Download button with dynamic content
    The content is generated using a callback when the button is clicked.
    """
    """This code is originally from 'ollik1', 
    posted in https://stackoverflow.com/questions/61708701/how-to-download-a-file-using-ipywidget-button
    on Aug 6, 2021
    """
    def __init__(self, filename: str, contents: Callable[[], str], **kwargs):
        super(DownloadButton, self).__init__(**kwargs)
        self.filename = filename
        self.contents = contents
        self.on_click(self.__on_click)

    def __on_click(self, b):
        contents: bytes = self.contents().encode('utf-8')
        b64 = base64.b64encode(contents)
        payload = b64.decode()
        digest = hashlib.md5(contents).hexdigest()  # bypass browser cache
        id = f'dl_{digest}'
        with out:
            display(HTML(f"""
                <html>
                <body>
                <a id="{id}" download="{self.filename}" href="data:text/csv;base64,{payload}" download>
                </a>

                <script>
                (function download() {{
                document.getElementById('{id}').click();
                }})()
                </script>

                </body>
                </html>
                """
            ))

In [16]:
def decompose_polygon_for_config(geom):
    """Extracts coords of a Polygon or Multipolygon and returns a list of x,y coords 
    (If MultiPolygon is a list of tuples, like:[(x,y),...]).
    INPUTS:
     - geom: Polygon or Multipoligon geometry
    OUTPUTS:
     - polxy: List of x,y of the decomposed Polygon (or list of tuples for Multipolygon).
     """
    if isinstance(geom, str):
        return '',''
    lines = geom.boundary
    if isinstance(lines, MultiLineString):
        mpolxy = []
        for pol in lines.geoms:
            mpolxy.append([list(r) for r in pol.xy])
    else:
        mpolxy = [list(r) for r in lines.xy]
    if isinstance(geom, MultiPolygon):
        polygon_type = 'multipolygon'
    else:
        polygon_type = 'singlepolygon'
        
    return mpolxy, polygon_type

def recompose_polygon_for_config(mpolxy, polygon_type):
    """From list of x,y coords (If MultiPolygon is a list of tuples, like:[(x,y),...]) 
    recompose a Polygon or Multipolygon.
    INPUTS:
     - polxy: List of x,y of the decomposed Polygon (or list of tuples for Multipolygon).
    OUTPUTS:
     - geom: Polygon or Multipoligon geometry
     """
    if (isinstance(mpolxy[0],tuple)) |(isinstance(mpolxy[0][0],list)):
        all_pol = []
        if polygon_type == 'multipolygon':
            for xx,yy in mpolxy:
                all_pol.append([(xi,yi) for xi,yi in zip(xx,yy)])
            geom = MultiPolygon([Polygon(s) for s in all_pol])
        elif polygon_type == 'singlepolygon':
            exterior = LinearRing([(xi,yi) for xi,yi in zip(mpolxy[0][0],mpolxy[0][1])])
            interiors = []
            for xx,yy in mpolxy[1::]:
                interiors.append(LinearRing([(xi,yi) for xi,yi in zip(xx,yy)]))
            geom = Polygon(exterior, interiors)
    else:
        geom = Polygon([(xi,yi) for xi,yi in zip(mpolxy[0],mpolxy[1])])
    return geom

def add_geometry(data = None, additional_geometry = None):
    if additional_geometry is None:
        return
    if data is None:
        if TOTAL_CONFIG:
            data = TOTAL_CONFIG.copy()
    else:
        data = data.copy()
    if isinstance(data['geometry'],str): # if it is a string it means it is empty
        new_geom = additional_geometry
    else:
        new_geom = unary_union([data['geometry'], additional_geometry])
    return new_geom

def chek_reply(reply):
    if (reply == 'y') | (reply == 'Y') | (reply is None):
        return True
    elif (reply == 'n') | (reply=='N'):
        return False
    else:
        print(f"\nYour reply was not clear, you inserted:'{reply}'.\nPlease insert 'Y' or press enter for YES, or 'n' for NO.\n")
        return None

def write_config(data = None, file = DEFAULT_CONFIG_FILE):
    if data is None:
        if TOTAL_CONFIG:
            data = TOTAL_CONFIG.copy()
    else:
        data = data.copy()
    answer = True
    if data['geometry']!='':
        data['geometry'], data['polygon_type'] = decompose_polygon_for_config(data['geometry'])
        
    # WRITE DOWN the config
    if answer:
        with open(file, 'w') as outfile:
            yaml.dump(data, outfile, default_flow_style=None)
    # activate download button
    download_shapefile() 

def read_config(config_file = None):    
    if config_file is None:
        config_file = DEFAULT_CONFIG_FILE
    with open(config_file) as src:
        dd = yaml.load(src,yaml.loader.FullLoader)
    if not isinstance(dd['geometry'], str):
        dd['geometry'] = recompose_polygon_for_config(dd['geometry'], dd['polygon_type'])
    return dd

def delete_all_additional_layers():
    dc.clear()
    for ll in m.layers:
        if (ll.name=='Selected country') | (ll.name=="Selected continent") | (ll.name=='old shp'):
            m.remove_layer(ll)
        elif ll.name == 'shp uploaded':
            m.remove_layer(ll)
            uploader.metadata.clear()
            uploader.data.clear()
            uploader.value.clear()
            uploader._counter = 0

In [17]:
if Path(DEFAULT_CONFIG_FILE).exists():
    TOTAL_CONFIG = read_config()
    set_default_values = True
    if TOTAL_CONFIG['geometry']=='':
        set_default_values = False
else:
    set_default_values = False

In [46]:
###########################################################################################################
# Create Map
###########################################################################################################
m = ipylf.Map(center=(28.6019917, 70.9121356), zoom=2, basemap=ipylf.basemaps.Esri.WorldTopoMap, scroll_wheel_zoom = True)

In [47]:
###########################################################################################################
# Add shapefile countries
###########################################################################################################
url_to_download    = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_map_units.zip"
file_path_location = Path.cwd() / url_to_download.split('/')[-1]
sbcm = subcountrymap(file_path_location = file_path_location, url_to_download = url_to_download)
countries = sbcm.shapefile.copy()
countries.index = countries.GEOUNIT
countries['name'] = countries.GEOUNIT
continent_shapefile = sbcm.continent_shapefile.copy()
all_shapes = pd.concat([countries, continent_shapefile])[['geometry','continent']]

In [48]:
### THIS IS THE LAYER TO SELECT COUNTRIES DIRECTLY ON MAP ###
geo_data = ipylf.GeoData(
    geo_dataframe=countries,
    style=default_style,
    hover_style= hover_style,
    name="Countries")
m.add_layer(geo_data)

html = ipywd.HTML("""Hover over a state""")
html.layout.margin = "0px 20px 20px 20px"
control = ipylf.WidgetControl(widget=html, position="topright")
m.add_control(control)

def update_html(feature, **kwargs):
    html.value = """
        <h4><b>{}</b></h4>
        <h5>Continent: {}</h5>
    """.format(
        feature["properties"]["NAME_EN"],
        feature["properties"]["continent"],
    )
    
def underline_shape(feature, **kwargs):
    selected_layer = ipylf.GeoJSON(
                        data=feature,
                        name="Selected country",
                        hover_style=hover_style,
                        style=click_style)
    m.add_layer(selected_layer)
    # write it in config file
    new_geom = unary_union(list(gpd.GeoDataFrame.from_features([feature]).geometry.values))
    TOTAL_CONFIG.update({'geometry': add_geometry(additional_geometry=new_geom)})
    write_config(TOTAL_CONFIG, file = DEFAULT_CONFIG_FILE)
    
geo_data.on_click(underline_shape)
geo_data.on_hover(update_html)

# # underline previous selection (if present in the configuraton file)
# if set_default_values:
#     geo_update = ipylf.GeoData(
#         geo_dataframe=gpd.GeoDataFrame(geometry = [TOTAL_CONFIG['geometry']]),
#         style=click_style,
#         name="old shp"
#     )
#     m.add_layer(geo_update)

In [49]:
m

Map(center=[28.6019917, 70.9121356], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [ ]:
### OTHER COMMANDS

In [ ]:
## 1. Cancel shapes command
button_clear_selection = ipywd.Button(
    value=False,
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to clear all the shapes',
    icon='trash', # (FontAwesome names without the `fa-` prefix)
    layout = ipywd.Layout(display='center',flex_flow='column',align_items='center',width='8mm')
)

def clear_selection_click(event):
    delete_all_additional_layers()
    # clear geometry from config
    TOTAL_CONFIG.update({'geometry':'','polygon_type':''})
    write_config(TOTAL_CONFIG, file = DEFAULT_CONFIG_FILE)

button_clear_selection.on_click(clear_selection_click)
bsc = ipylf.WidgetControl(widget=button_clear_selection, position = 'topleft')
m.add_control(bsc)

In [ ]:
## 2. Upload shapes command
uploader = ipywd.FileUpload(
    multiple = True,
    description='Upload shp',
    style = {'description_width': 'initial'},
    tooltip='select shapefile to load from local',
    layout = ipywd.Layout( display='center', flex_flow='column', align_items='center', width='18mm')
)

def read_shape_from_uploader(event):
    """Load the shapefile selected into memory and show it in the map"""
    list_allowed_extensions = ['zip','geojson','shp','kml', 'kmz', 'gpkg']
    for kk, val in event.new.items():
        for kk1, val1 in val.items():
            if kk1 =='metadata':
                for kk2,val2 in val1.items():
                    if kk2=='name':
                        extension = val2.split('.')[-1]
                        # check the extension of the file
                        if extension in list_allowed_extensions:
                            # read the uploaded file
                            shp = gpd.read_file(BytesIO(val['content']))
                            # DRAW it on the map
                            geo_update = ipylf.GeoData(
                                geo_dataframe=shp,
                                style=click_style,
                                name="shp uploaded"
                            )
                            m.add_layer(geo_update)
                            # write it in config file
                            new_geom = unary_union(list(shp.geometry.values))
                            TOTAL_CONFIG.update({'geometry': add_geometry(additional_geometry=new_geom)})
                            write_config(TOTAL_CONFIG, file = DEFAULT_CONFIG_FILE)
                        else:
                            print(f"file {val2} with extension {extension} NOT allowed.\nPlease upload a file with one of the following extension {list_allowed_extensions}\n")

uploader.observe(read_shape_from_uploader, 'value')
up = ipylf.WidgetControl(widget=uploader, position = 'topright')
m.add_control(up)

In [45]:
## 3. Download shapes command
db_shp = DownloadButton(
    filename='example.geojson', contents=lambda: None, 
    disabled = True, 
    description='Download shp',
    style = {'description_width': 'initial'},
    tooltip = 'download shp in GeoJSON format',
    icon    = 'download', 
)

def download_shapefile(filename_dwnd = None):
    if not isinstance(TOTAL_CONFIG['geometry'], str):
        tt = gpd.GeoDataFrame(geometry = [TOTAL_CONFIG['geometry']])
        #DOWNLOAD BUTTON APPEARS!
        if filename_dwnd is not None:
            db_shp.filename = filename_dwnd
        db_shp.contents = lambda: tt.to_json()
        db_shp.disabled = False
    else:
        db_shp.disabled = True

In [ ]:
## 4. Refresher
refresher = ipywd.Button(
    value=False,
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to refresh (dissolve shapes into one)',
    icon='object-ungroup', # (FontAwesome names without the `fa-` prefix)
    layout = ipywd.Layout(display='center',flex_flow='column',align_items='center',width='8mm')
)

def refresh_map(event):
    """Draw again the geometries but this time dissolved together"""
    delete_all_additional_layers()
    # DRAW it on the map
    geo_update = ipylf.GeoData(
        geo_dataframe=gpd.GeoDataFrame(geometry = [TOTAL_CONFIG['geometry']]),
        style=click_style,
        name="shp uploaded"
    )
    m.add_layer(geo_update)

refresher.on_click(refresh_map)
rf = ipylf.WidgetControl(widget=refresher, position = 'topleft')
m.add_control(rf)

In [2]:
dc = ipylf.DrawControl(
    edit=True,
    remove=False, #True,
    polygon = {"shapeOptions": click_style},
    marker={},
    rectangle={"shapeOptions": click_style},
    polyline={},
    circle={},
    circlemarker={},
)

def handle_draw(target, action, geo_json):
    if (action == 'created') | (action =='edited'):
        # write it in config file
        new_geom = unary_union(list(gpd.GeoDataFrame.from_features([geo_json]).geometry.values))
        TOTAL_CONFIG.update({'geometry': add_geometry(additional_geometry=new_geom)})
        write_config(TOTAL_CONFIG, file = DEFAULT_CONFIG_FILE)
    elif action == 'deleted':
        delete_all_additional_layers()
        # clear geometry from config
        TOTAL_CONFIG.update({'geometry':'','polygon_type':''})
        write_config(TOTAL_CONFIG, file = DEFAULT_CONFIG_FILE)

dc.on_draw(handle_draw)
m.add_control(dc)

Map(center=[28.6019917, 70.9121356], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [ ]:
## 3. Download shapes command - draw
dwnshp = ipylf.WidgetControl(widget=db_shp, position = 'topright')
m.add_control(dwnshp)

## 4. Fullscreen - draw
fs_control = ipylf.FullScreenControl()
m.add_control(fs_control)

In [ ]:
###########################################################################################################
# Add dropdown menu with continents/countries
###########################################################################################################
cmbox1 = ipywd.Combobox(options=list(continent_shapefile.index),
                        placeholder='CONTINENT',
                        description=" ",
                        layout = {'width': 'max-content', 'margin' :'-1.5mm 1mm 1mm -22mm'}
#                         layout=ipywd.Layout(display='flex', flex_flow='column', align_items='initial', width='30mm')
                 )
list_countries = list(countries.index)
cmbox2 = ipywd.Combobox(options=list(list_countries),
                    placeholder='COUNTRY',
                    description=" ",
                    indent=False,
                    layout = {'width': 'max-content', 'margin' :'-1.5mm 1mm 1mm -22mm'}
#                     layout=ipywd.Layout(display='', flex = '1 1 10%', flex_flow='column', align_items='initial', width='30mm', height = '8mm')
                 )

def update_cmbox2_and_plot_cmbox1(continent):
    # UPDATE
    list_countries = list(countries[countries.continent==continent].index.values) # list(sbcm.shapefile[sbcm.shapefile.continent ==continent].GEOUNIT.values)
    if len(list_countries)>0:
        cmbox2.options = list_countries
        #PLOT
        #continent = continent.upper()
        rr = all_shapes[all_shapes.index == continent]
        if len(rr)>0:
            # DRAW it on the map
            geo_update = ipylf.GeoData(
                geo_dataframe=rr,
                style=click_style,
                name="Selected continent",
            )
            m.add_layer(geo_update)
            # write it in config file
            new_geom = unary_union(list(rr.geometry.values))
            TOTAL_CONFIG.update({'geometry': add_geometry(additional_geometry=new_geom)})
            write_config(TOTAL_CONFIG, file = DEFAULT_CONFIG_FILE)           

def remove_continents():
    """Function to Remove only the continent layer"""
    for ll in m.layers:
        if ll.name=="Selected continent":
            m.remove_layer(ll)
            
def plot_cmbox2(country):
    #extract selected country/continent
    rr = all_shapes[all_shapes.index == country]
    if len(rr)>0:
        # remove continent - for now we remove th entire shape
        clear_selection_click(None)
        # DRAW it on the map
        geo_update = ipylf.GeoData(
            geo_dataframe=rr,
            style=click_style,
            name="Selected country",
        )
        m.add_layer(geo_update)   
        # write it in config file
        new_geom = unary_union(list(rr.geometry.values))
        TOTAL_CONFIG.update({'geometry': add_geometry(additional_geometry=new_geom)})
        write_config(TOTAL_CONFIG, file = DEFAULT_CONFIG_FILE)

inte1 = ipywd.interactive(update_cmbox2_and_plot_cmbox1, continent = cmbox1)
inte2 = ipywd.interactive(plot_cmbox2, country = cmbox2)
win_continents = ipylf.WidgetControl(widget=inte1, position = 'topright')
m.add_control(win_continents)
win_countries = ipylf.WidgetControl(widget=inte2, position = 'topright')
m.add_control(win_countries)